In [1]:
# coding: utf8

# Calcul des Coformes pour le lexique complet

In [2]:
# -*- coding: utf8 -*-
import pandas as pd
import pickle, glob
import itertools as it
import networkx as nx
import random as rd
import numpy as np
import matplotlib,math
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import ipywidgets as widgets

In [31]:
repHDR="/Users/gilles/ownCloud/Recherche/Boye/HDR/Memoire/figs/"
filePrefix="/Volumes/gilles/Transfert/Copies-iMac-GB/2015-Data/lexique3/"

# Gestion des espaces thématiques

In [4]:
etCells=[
    ["pi1P","pi2P"]+["ii"+p+n for p in "123" for n in "SP"],
    ["pi3P"],
    ["pi"+p+"S" for p in "123"],
    ["pP"],
    ["pI2S"],
    ["pI1P","pI2P"],
    ["ps"+p+"S" for p in "123"]+["ps3P"],
    ["ps1P","ps2P"],
    ["inf"],
    ["fi"+p+n for p in "123" for n in "SP"]+["pc"+p+n for p in "123" for n in "SP"],
    ["ai"+p+n for p in "123" for n in "SP"]+["is"+p+n for p in "123" for n in "SP"],
    ["pp"+g+n for g in "MF" for n in "SP"],
        ]

latex2RGB={"orange":(240,134,51),
           "brown!50":(218,192,163),
           "brown":(182,130,75),
           "blue!10":(230,230,253),
           "yellow!50":(255,246,164),
           "blue!25":(191,192,250),
           "teal!50":(141,190,190),
           "blue!50":(128,129,247),
           "cyan!50":(133,198,227),
           "lime":(204,252,81),
           "pink":(246,194,193),
           "magenta!50":(221,151,180),
          }
etRGB=[
        (240,134,51),
        (218,192,163),
        (182,130,75),
        (230,230,253),
        (255,246,164),
        (191,192,250),
        (141,190,190),
        (128,129,247),
        (133,198,227),
        (204,252,81),
        (246,194,193),
        (221,151,180),
       ]
etRGBx=[(r/255.,g/255.,b/255.) for r,g,b in etRGB]

cellColors={}
for nEt,et in enumerate(etCells):
    for c in et:
        cellColors[c]=etRGBx[nEt]

personnes=[p+n for n in "SP" for p in "123" ]
cases=[t+p for t in "pi ii fi pc ps ai is".split(" ") for p in personnes]
cases+="pI2S pI1P pI2P inf pP".split(" ")
cases+=["pp"+g+n for g in "MF" for n in "SP"]
#cases

In [5]:
def colorerXTicks(gAX):
    xlabels=gAX.get_xticklabels()
    for xlabel in xlabels:
        xtext=xlabel.get_text()
        xlabel.set_backgroundcolor(cellColors[xtext])
        xlabel.set_family("monospace")


In [6]:
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.set_palette("hls")
sns.set_color_codes("muted")
#sns.palplot(sns.color_palette("hls", 8))
#sns.palplot(sns.color_palette("husl", 8))

In [7]:
%matplotlib inline
#from __future__ import unicode_literals

In [8]:
lexiquePrefix="MGC-171229"

# Défectifs de Boyé(2000)

In [9]:
boyeDefectifs=[u"abstraire", u"accroire", u"adirer", u"advenir", u"apparoir", u"assavoir", u"avenir",
               u"becter", u"béer", u"bienvenir", u"braire", u"bruire", u"chaloir", u"choir", u"clore", 
               u"comparoir", u"contrefoutre", u"courre", u"déchoir", u"déclore", u"dépourvoir", 
               u"discontinuer", u"dissoudre", u"distraire", u"douer", u"échoir", u"éclore", u"enclore", 
               u"endêver", u"ensuivre", u"ester", u"extraire", u"faillir", u"férir", u"forclore", 
               u"forfaire", u"foutre", u"frire", u"gésir", u"huir", u"impartir", u"issir", u"malfaire", 
               u"méfaire", u"messeoir", u"moufeter", u"moufter", u"mourir", u"oindre", u"ouïr", u"paître", 
               u"parfaire", u"poindre", u"quérir", u"raire", u"rassir", u"ravoir", u"reclure", u"revaloir", 
               u"saillir", u"seoir", u"sourdre", u"soustraire", u"stupéfaire", u"surfaire", u"titre", 
               u"traire", u"urger"]


# Lecture de Verbes3
Verbes3 contient les fréquences et les probabilités calculées pour répartir les fréquences brutes de Lexique3 entre les formes homographes.

In [10]:
lexique=pd.read_pickle("/Users/gilles/ownCloud/Recherche/Boye/HDR/Data/Samples/"+lexiquePrefix+'-Verbes3.pkl')

## Calcul du nombre de cases à remplir par lexème

In [11]:
nbCases=lexique.groupby(by=["lexeme","case"],as_index=False)["phono"].count()
nbFormes=nbCases.groupby(by=["lexeme"],as_index=True)[["case"]].count()
if len(nbFormes.columns)==2:
    nbFormes.columns=["lexeme","nbFormes"]
else:
    nbFormes.columns=["nbFormes"]

In [12]:
dictNbFormes=nbFormes.to_dict()["nbFormes"]
dictNbFormes

{u'anordir': 48,
 u'\xe9ditionner': 51,
 u'gauchir': 51,
 u'd\xe9nazifier': 51,
 u'r\xe9installer': 51,
 u'caner': 51,
 u'tancer': 51,
 u'devenir': 51,
 u'orthographier': 51,
 u'superposer': 51,
 u'affectionner': 51,
 u'verser': 51,
 u'muloter': 48,
 u'd\xe9gommer': 51,
 u'accouer': 51,
 u'd\xe9verser': 51,
 u'pleuvoter': 17,
 u'spiritualiser': 51,
 u'empoissonner': 51,
 u'persister': 48,
 u'd\xe9ganter': 51,
 u'juda\xefser': 51,
 u'commenter': 51,
 u'concasser': 51,
 u'bossuer': 51,
 u'\xe9peurer': 51,
 u'maquereauter': 51,
 u'boursicoter': 48,
 u'aposter': 51,
 u'chauffer': 51,
 u'poiler': 51,
 u'miniaturer': 51,
 u'd\xe9payser': 51,
 u'pencher': 51,
 u'poivrer': 51,
 u'\xe9vider': 51,
 u'foisonner': 48,
 u'pr\xe9luder': 48,
 u'moucharder': 51,
 u'muer': 51,
 u'revouloir': 51,
 u'd\xe9l\xe9aturer': 51,
 u'effaroucher': 51,
 u'd\xe9sulfurer': 51,
 u'd\xe9valoriser': 51,
 u'grossir': 51,
 u'fusionner': 51,
 u'confisquer': 51,
 u'reporter': 51,
 u'susurrer': 51,
 u'd\xe9sincruster': 51,

In [38]:
formesLex3=lexique[lexique["freq"]>=1E4]["lexeme case phono".split(" ")]
nbFormes=formesLex3.count()["case"]
nbTokens=150000000

In [39]:
formesLex3["tir1"]=1

In [40]:
formesLex3.reset_index(drop=True,inplace=True)

In [41]:
formesLex3.to_pickle(path=filePrefix+"Longitudinal-Lexique3-00-T%d-F%d.pkl"%(nbTokens,nbFormes))